# Basic tutorial: Question answer
#### Author: Matteo Caorsi

This short tutorial provides you with the basic functioning of *giotto-deep* API.

The example described in this tutorial is the one of question answer.

The main steps of the tutorial are the following:
 1. creation of a dataset
 2. creation of a model
 3. define metrics and losses
 4. train the model
 5. extract some features of the network

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np

import torch
from torch import nn

from gdeep.models import FFNet

from gdeep.visualisation import persistence_diagrams_of_activations

from torch.utils.tensorboard import SummaryWriter
from gdeep.data.datasets import DatasetBuilder
from gdeep.trainer import Trainer

from gtda.diagrams import BettiCurve

from gtda.plotting import plot_betti_surfaces


# Initialize the tensorboard writer

In order to analyse the reuslts of your models, you need to start tensorboard.
On the terminal, move inside the `/example` folder. There run the following command:

```
tensorboard --logdir=runs
```

Then go [here](http://localhost:6006/) after the training to see all the visualisation results.

In [ ]:
writer = SummaryWriter()


# Create your dataset

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler


bd = DatasetBuilder(name="SQuAD2", convert_to_map_dataset=True)
ds_tr_str, ds_val_str, ds_ts_str = bd.build()


The dataset contains a context and a question whose answer can be found within that context. The correct answer as well as the starting characters are also provided.

In [ ]:
print("Before preprocessing: \n", ds_tr_str[0])


## Required preprocessing

Neural networks cannot direcly deal with strings. We have first to preprocess the dataset in three main ways:
 1. Tokenise the strings into its words
 2. Build a vocabulary out of these words
 3. Embed each word into a vector, so that each sentence becomes a list of vectors

The first two steps are performed by the `PreprocessTextQA`. The embedding will be added directly to the model.

In [ ]:
from gdeep.data import PreprocessingPipeline
from gdeep.data import TransformingDataset
from gdeep.data.preprocessors import Normalization, TokenizerQA
from gdeep.data.datasets import DataLoaderBuilder


tokenizer = TokenizerQA()

# in case you need to combine multiple preprocessing:
# ppp = PreprocessingPipeline(((PreprocessTextData(), IdentityTransform(), TextDataset),
#                             (Normalisation(), IdentityTransform(), BasicDataset)))


tokenizer.fit_to_dataset(ds_tr_str)
transformed_textds = tokenizer.attach_transform_to_dataset(ds_tr_str)

transformed_textts = tokenizer.attach_transform_to_dataset(
    ds_val_str
)  # this has been fitted on the train set!

print("After the preprocessing: \n", transformed_textds[0])

# the only part of the training/test set we are interested in
train_indices = list(range(64 * 2))
test_indices = list(range(64 * 1))

dl_tr2, dl_ts2, _ = DataLoaderBuilder((transformed_textds, transformed_textts)).build(
    (
        {"batch_size": 16, "sampler": SubsetRandomSampler(train_indices)},
        {"batch_size": 16, "sampler": SubsetRandomSampler(test_indices)},
    )
)


## Define and train your model

The model for QA shall accept as input the context and the question and return the probabilities for the initial and final token of the answer in the input context. The output than, is a pair of logits.

In [ ]:
from torch.nn import Transformer
from torch.optim import Adam, SparseAdam, SGD
import copy

# my simple transformer model
class QATransformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, embed_dim):
        super(QATransformer, self).__init__()
        self.transformer = Transformer(
            d_model=embed_dim,
            nhead=2,
            num_encoder_layers=1,
            num_decoder_layers=1,
            dim_feedforward=512,
            dropout=0.1,
        )
        self.embedding_src = nn.Embedding(src_vocab_size, embed_dim, sparse=True)
        self.embedding_tgt = nn.Embedding(tgt_vocab_size, embed_dim, sparse=True)
        self.generator = nn.Linear(embed_dim, 2)

    def forward(self, X):
        # print(X.shape)
        src = X[:, 0, :]
        tgt = X[:, 1, :]
        # print(src.shape, tgt.shape)
        src_emb = self.embedding_src(src)
        tgt_emb = self.embedding_tgt(tgt)
        # print(src_emb.shape, tgt_emb.shape)
        self.outs = self.transformer(src_emb, tgt_emb)
        # print(outs.shape)
        logits = self.generator(self.outs)
        # print(logits.shape)
        # out = torch.topk(logits, k=1, dim=2).indices.reshape(-1,44)
        # print(out, out.shape)
        return logits

    def __deepcopy__(self, memo):
        """this is needed to make sure that the 
        non-leaf nodes do not
        interfere with copy.deepcopy()
        """
        cls = self.__class__
        result = cls.__new__(cls)
        memo[id(self)] = result
        for k, v in self.__dict__.items():
            setattr(result, k, copy.deepcopy(v, memo))
        return result

    def encode(self, src, src_mask):
        """this method is used only at the inference step"""
        return self.transformer.encoder(self.embedding_src(src), src_mask)

    def decode(self, tgt, memory, tgt_mask):
        """this method is used only at the inference step"""
        return self.transformer.decoder(self.embedding_tgt(tgt), memory, tgt_mask)


In [ ]:
vocab_size = 1500000

src_vocab_size = vocab_size
tgt_vocab_size = vocab_size
emb_size = 64

model = QATransformer(src_vocab_size, tgt_vocab_size, emb_size)
print(model)


## Define the loss function

This loss function is a adapted version of the Cross Entropy for the trnasformer architecture.

In [ ]:
def loss_fn(output_of_network, label_of_dataloader):
    # print(output_of_network.shape, label_of_dataloader.shape)
    tgt_out = label_of_dataloader
    # print(tgt_out)
    logits = output_of_network
    cel = nn.CrossEntropyLoss()
    return cel(logits, tgt_out)


In [ ]:
# prepare a pipeline class with the model, dataloaders loss_fn and tensorboard writer
pipe = Trainer(model, (dl_tr2, dl_ts2), loss_fn, writer)

# train the model
pipe.train(SGD, 3, False, {"lr": 0.01}, {"batch_size": 16})


## Answering questions!

Here we have a question and its associated context:

In [ ]:
bb = next(iter(ds_val_str))
bb[:2]



Get the vocabulary and numericize the question and context to then input both to the model.

In [ ]:
voc = tokenizer.vocabulary
context = tokenizer.tokenizer(bb[0])
question = tokenizer.tokenizer(bb[1])

# get the indexes in the vocabulary of the tokens
context_idx = torch.tensor(list(map(voc.__getitem__, context)))
question_idx = torch.tensor(list(map(voc.__getitem__, question)))


In [ ]:
aa = next(iter(dl_tr2))
length_to_pad = aa[0].shape[-1]
pad_fn = lambda item: torch.cat(
    [item, tokenizer.pad_item * torch.ones(length_to_pad - item.shape[0])]
).to(torch.long)

# these tansors are ready to be fitted into the model
context_ready_for_model = pad_fn(context_idx)
question_ready_for_model = pad_fn(question_idx)


Put the two tensors of context and question together and input them to the model

In [ ]:
inp = (
    torch.stack((context_ready_for_model, question_ready_for_model))
    .reshape(1, *aa[0].shape[1:])
    .long()
)
out = model(inp)


The output corresponds to the digits for the start and end tokens of the answer. It is now time to extract them with `torch.argmax`

In [ ]:
answer_idx = torch.argmax(out, dim=1)

try:
    if answer_idx[0][1] > answer_idx[0][0]:
        print(
            "The model proposes: '",
            context[answer_idx[0][0] : answer_idx[0][1]],
            "...'",
        )
    else:
        print("The model proposes: '", context[answer_idx[0][0]], "...'")
except IndexError:
    print("The model was not able to find the answer.")
print("The actual answer was: '" + bb[2][0] + "'")


# Extract inner data from your models

In [ ]:
from gdeep.models import ModelExtractor

me = ModelExtractor(pipe.model, loss_fn)

lista = me.get_layers_param()

for k, item in lista.items():
    print(k, item.shape)


In [ ]:
DEVICE = torch.device("cpu")
x = next(iter(dl_tr2))[0]
pipe.model.eval()
pipe.model(x.to(DEVICE))

list_activations = me.get_activations(x)
len(list_activations)


In [ ]:
x = next(iter(dl_tr2))[0][0]
if x.dtype is not torch.int64:
    res = me.get_decision_boundary(x, n_epochs=1)
    res.shape


In [ ]:
x, target = next(iter(dl_tr2))
if x.dtype is torch.float:
    for gradient in me.get_gradients(x, target=target)[1]:
        print(gradient.shape)


# Visualise activations and other topological aspects of your model

In [ ]:
from gdeep.visualisation import Visualiser

vs = Visualiser(pipe)

vs.plot_data_model()
# vs.plot_activations(x)
# vs.plot_persistence_diagrams(x)
